<a href="https://colab.research.google.com/github/jiyeonjin/0624_new/blob/main/0724_roboflow_%EC%A0%84%EC%9D%B4%ED%95%99%EC%8A%B5_%EC%83%98%ED%94%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

강사님이 보내주신 전이학습 샘플

In [ ]:
# 1. 필요한 라이브러리 설치
!pip install yt-dlp roboflow ultralytics opencv-python

import cv2
import os
from roboflow import Roboflow
import numpy as np
from ultralytics import YOLO
import yt_dlp   # 차선 감지 모델을 위한 YouTube 영상 처리
# Roboflow에서 이미 훈련된 모델을 불러와서 사용
#영상에서 프레임을 추출하여 **추론(inference)**만 수행
import cv2
import os
from roboflow import Roboflow
import yt_dlp

def load_lane_detection_model():
    """
    차선 감지 모델 로드
    """
    rf = Roboflow(api_key="JwvZQEBhBR5uPrwepqQW")
    #project = rf.workspace().project("0722_labeling-usrpl/1")
    project = rf.workspace().project("0722_labeling-usrpl")
    model = project.version(1).model
    print("🛣️ 차선 감지 모델 로드 완료!")
    return model



def download_driving_video(url, output_path="./"):
    """
    드라이빙 영상 다운로드
    """
    ydl_opts = {
        'format': 'best[height<=720]',
        'outtmpl': os.path.join(output_path, 'driving_video.%(ext)s'),
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            ydl.download([url])
            print("✅ 드라이빙 영상 다운로드 완료!")
            return True
        except Exception as e:
            print(f"❌ 다운로드 실패: {e}")
            return False

def detect_lanes_in_video(video_path, model, output_path="lane_detection_result.mp4"):
    """
    영상에서 차선 감지
    """
    cap = cv2.VideoCapture(video_path)

    # 영상 정보
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"🎬 영상 정보: {width}x{height}, {fps}fps, {total_frames}프레임")

    # 출력 설정
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    lane_detections = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # 매 3프레임마다 차선 감지 (더 자주)
        if frame_count % 2 == 0:
            try:
                temp_img_path = "temp_frame.jpg"
                cv2.imwrite(temp_img_path, frame)

                # 차선 감지 (낮은 신뢰도)
                prediction = model.predict(temp_img_path, confidence=30, overlap=50)
                predictions = prediction.json()['predictions']

                frame_lanes = len(predictions)
                lane_detections += frame_lanes

                if frame_lanes > 0:
                    print(f"🛣️ 프레임 {frame_count}: {frame_lanes}개 차선 감지")

                # 차선 그리기
                for lane in predictions:
                    x1 = int(lane['x'] - lane['width']/2)
                    y1 = int(lane['y'] - lane['height']/2)
                    x2 = int(lane['x'] + lane['width']/2)
                    y2 = int(lane['y'] + lane['height']/2)

                    # 차선은 보라색으로 표시
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 255), 3)

                    # 라벨
                    label = f"Lane: {lane['confidence']:.2f}"
                    cv2.rectangle(frame, (x1, y1-30), (x1+150, y1), (255, 0, 255), -1)
                    cv2.putText(frame, label, (x1+5, y1-8),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

                if os.path.exists(temp_img_path):
                    os.remove(temp_img_path)

            except Exception as e:
                print(f"❌ 프레임 {frame_count} 처리 오류: {e}")

        out.write(frame)

        # 진행상황
        if frame_count % 150 == 0:
            progress = (frame_count / total_frames) * 100
            print(f"📊 진행률: {progress:.1f}% (총 차선 감지: {lane_detections}개)")

    cap.release()
    out.release()

    print(f"✅ 완료! 총 {lane_detections}개 차선 감지")
    print(f"🎥 결과: {output_path}")

def main_lane_detection():
    """
    차선 감지 메인 함수
    """
    # 추천 YouTube URL들 (드라이빙 영상)
    driving_urls = [
        "https://www.youtube.com/watch?v=_CGb4GYHTvg",  # 기존 URL
        # 더 나은 드라이빙 영상이 있다면 교체하세요
    ]

    youtube_url = driving_urls[0]

    print("🚗 드라이빙 영상 다운로드 시작...")
    if download_driving_video(youtube_url):

        # 다운로드된 파일 찾기
        video_files = [f for f in os.listdir('.') if f.startswith('driving_video')]
        if video_files:
            video_path = video_files[0]
            print(f"📁 영상 파일: {video_path}")

            print("🛣️ 차선 감지 모델 로드 중...")
            try:
                model = load_lane_detection_model()

                print("🔍 차선 감지 시작...")
                detect_lanes_in_video(video_path, model, "lane_detection_result.mp4")

                print("🎉 차선 감지 완료!")
                print("📺 결과 영상: lane_detection_result.mp4")

            except Exception as e:
                print(f"❌ 모델 로드 실패: {e}")
                print("🔑 API 키를 확인해주세요!")
        else:
            print("❌ 다운로드된 영상 파일을 찾을 수 없습니다.")

def test_with_webcam_url():
    """
    Roboflow Visualize 페이지의 'Paste YouTube or Image URL' 기능 사용
    """
    print("🌐 웹 인터페이스 테스트:")
    print("1. Roboflow Visualize 페이지에서")
    print("2. 'Paste YouTube or Image URL' 입력창에")
    print("3. YouTube URL 붙여넣기")
    print("4. 차선이 잘 감지되는지 확인")

# 실행
if __name__ == "__main__":
    print("🛣️ 차선 감지 모드로 변경!")
    print("=" * 50)
    print("💡 이 모델은 차선(lane)을 감지하는 모델입니다.")
    print("📹 드라이빙 영상이나 도로 영상에서 가장 잘 작동합니다.")
    print()
    print("🚀 실행 방법:")
    print("1. API 키 입력")
    print("2. main_lane_detection() 실행")
    print("3. 또는 Roboflow 웹에서 test_with_webcam_url() 방법 시도")
    print("=" * 50)
    main_lane_detection()

유튜브 url 업로드 해서 라벨링 한 영상 다운받기

In [2]:
# 1. 필요한 라이브러리 설치
!pip install yt-dlp roboflow opencv-python

import cv2
import os
from roboflow import Roboflow
import numpy as np
import yt_dlp

def load_object_detection_model(api_key, workspace_name, project_name, version=1):
    """
    Roboflow에서 객체 감지 모델 로드
    """
    try:
        rf = Roboflow(api_key=api_key)
        project = rf.workspace(workspace_name).project(project_name)
        model = project.version(version).model
        print(f"🎯 Roboflow 객체 감지 모델 로드 완료!")
        print(f"📁 프로젝트: {workspace_name}/{project_name} v{version}")
        return model
    except Exception as e:
        print(f"❌ 모델 로드 실패: {e}")
        print("🔑 API 키, 워크스페이스명, 프로젝트명을 확인해주세요!")
        return None

def download_youtube_video(url, output_path="./"):
    """
    YouTube 영상 다운로드
    """
    ydl_opts = {
        'format': 'best[height<=720]',
        'outtmpl': os.path.join(output_path, 'input_video.%(ext)s'),
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            ydl.download([url])
            print("✅ YouTube 영상 다운로드 완료!")
            return True
        except Exception as e:
            print(f"❌ 다운로드 실패: {e}")
            return False

def detect_objects_in_video(video_path, model, output_path="roboflow_detection_result.mp4"):
    """
    영상에서 객체 감지 (Roboflow 모델 사용)
    """
    cap = cv2.VideoCapture(video_path)

    # 영상 정보
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"🎬 영상 정보: {width}x{height}, {fps}fps, {total_frames}프레임")

    # 출력 설정
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    total_detections = 0
    detected_classes = set()

    # 색상 설정 (다양한 객체를 위한 색상 팔레트)
    colors = [
        (255, 0, 0),     # 빨강
        (0, 255, 0),     # 초록
        (0, 0, 255),     # 파랑
        (255, 255, 0),   # 노랑
        (255, 0, 255),   # 마젠타
        (0, 255, 255),   # 시안
        (255, 165, 0),   # 주황
        (128, 0, 128),   # 보라
        (255, 192, 203), # 핑크
        (0, 128, 0),     # 어두운 초록
    ]

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # 매 3프레임마다 객체 감지 (성능 최적화)
        if frame_count % 3 == 0:
            try:
                # 임시 이미지 저장
                temp_img_path = "temp_frame.jpg"
                cv2.imwrite(temp_img_path, frame)

                # Roboflow로 객체 감지
                prediction = model.predict(temp_img_path, confidence=40, overlap=50)
                predictions = prediction.json()['predictions']

                frame_detections = len(predictions)
                total_detections += frame_detections

                if frame_detections > 0:
                    print(f"🎯 프레임 {frame_count}: {frame_detections}개 객체 감지")

                # 감지된 객체들 그리기
                for i, obj in enumerate(predictions):
                    # 바운딩 박스 좌표
                    x1 = int(obj['x'] - obj['width']/2)
                    y1 = int(obj['y'] - obj['height']/2)
                    x2 = int(obj['x'] + obj['width']/2)
                    y2 = int(obj['y'] + obj['height']/2)

                    # 클래스명과 신뢰도
                    class_name = obj['class']
                    confidence = obj['confidence']
                    detected_classes.add(class_name)

                    # 색상 선택 (클래스별로 일관된 색상)
                    color_idx = hash(class_name) % len(colors)
                    color = colors[color_idx]

                    # 바운딩 박스 그리기
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

                    # 라벨 텍스트
                    label = f"{class_name}: {confidence:.2f}"
                    label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]

                    # 라벨 배경
                    cv2.rectangle(frame, (x1, y1 - label_size[1] - 10),
                                (x1 + label_size[0], y1), color, -1)

                    # 라벨 텍스트
                    cv2.putText(frame, label, (x1, y1 - 5),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

                # 임시 파일 삭제
                if os.path.exists(temp_img_path):
                    os.remove(temp_img_path)

            except Exception as e:
                print(f"❌ 프레임 {frame_count} 처리 오류: {e}")

        out.write(frame)

        # 진행상황 표시
        if frame_count % 150 == 0:
            progress = (frame_count / total_frames) * 100
            print(f"📊 진행률: {progress:.1f}% (총 객체 감지: {total_detections}개)")

    cap.release()
    out.release()

    print(f"✅ 완료! 총 {total_detections}개 객체 감지")
    print(f"🏷️ 감지된 클래스들: {', '.join(sorted(detected_classes))}")
    print(f"🎥 결과 영상: {output_path}")

def main_roboflow_detection():
    """
    Roboflow 객체 감지 메인 함수
    """
    # ========== 설정 부분 ==========
    # 여기에 API 키만 입력하세요
    API_KEY = "JwvZQEBhBR5uPrwepqQW"  # API 키 설정 완료
    WORKSPACE = "jiyeonjin"        # 워크스페이스명
    PROJECT = "0722_labeling-usrpl" # 프로젝트명
    VERSION = 1                    # 모델 버전

    youtube_url = "https://www.youtube.com/watch?v=_CGb4GYHTvg"

    # API 키 확인
    if API_KEY == "YOUR_API_KEY_HERE":
        print("❌ API 키를 입력해주세요!")
        print("코드에서 API_KEY, WORKSPACE, PROJECT 변수를 수정하세요.")
        return

    print("📺 YouTube 영상 다운로드 시작...")
    if download_youtube_video(youtube_url):

        # 다운로드된 파일 찾기
        video_files = [f for f in os.listdir('.') if f.startswith('input_video')]
        if video_files:
            video_path = video_files[0]
            print(f"📁 영상 파일: {video_path}")

            print("🎯 Roboflow 모델 로드 중...")
            model = load_object_detection_model(API_KEY, WORKSPACE, PROJECT, VERSION)

            if model is not None:
                print("🔍 객체 감지 시작...")
                detect_objects_in_video(video_path, model, "roboflow_detection_result.mp4")

                print("🎉 객체 감지 완료!")
                print("📺 결과 영상: roboflow_detection_result.mp4")

            else:
                print("❌ 모델 로드에 실패했습니다.")
        else:
            print("❌ 다운로드된 영상 파일을 찾을 수 없습니다.")
    else:
        print("❌ YouTube 영상 다운로드에 실패했습니다.")

def test_roboflow_single_frame():
    """
    단일 프레임으로 Roboflow 모델 테스트
    """
    # ========== 설정 부분 ==========
    API_KEY = "JwvZQEBhBR5uPrwepqQW"  # API 키 설정 완료
    WORKSPACE = "jiyeonjin"        # 워크스페이스명
    PROJECT = "0722_labeling-usrpl" # 프로젝트명
    VERSION = 1

    youtube_url = "https://www.youtube.com/watch?v=_CGb4GYHTvg"

    print("🖼️ 단일 프레임 테스트 모드")

    if download_youtube_video(youtube_url):
        video_files = [f for f in os.listdir('.') if f.startswith('input_video')]
        if video_files:
            cap = cv2.VideoCapture(video_files[0])
            ret, frame = cap.read()
            if ret:
                cv2.imwrite("test_frame.jpg", frame)

                model = load_object_detection_model(API_KEY, WORKSPACE, PROJECT, VERSION)
                if model:
                    try:
                        prediction = model.predict("test_frame.jpg", confidence=40)
                        predictions = prediction.json()['predictions']

                        print(f"✅ {len(predictions)}개 객체 감지됨!")
                        for obj in predictions:
                            print(f"  - {obj['class']}: {obj['confidence']:.2f}")

                        # 결과 이미지 URL 표시 (있는 경우)
                        if hasattr(prediction, 'save'):
                            prediction.save("test_result.jpg")
                            print("✅ 테스트 결과 저장: test_result.jpg")

                    except Exception as e:
                        print(f"❌ 예측 실패: {e}")
            cap.release()

def setup_api_credentials(api_key, workspace, project, version=1):
    """
    API 인증 정보 설정 함수
    """
    global API_KEY, WORKSPACE, PROJECT, VERSION
    API_KEY = api_key
    WORKSPACE = workspace
    PROJECT = project
    VERSION = version

    print("✅ API 인증 정보 설정 완료!")
    print(f"🔑 API Key: {api_key[:10]}...")
    print(f"🏢 Workspace: {workspace}")
    print(f"📁 Project: {project}")
    print(f"🔢 Version: {version}")

# 실행
if __name__ == "__main__":
    print("🎯 Roboflow YouTube 영상 객체 감지 시스템")
    print("=" * 60)
    print("💡 Roboflow 사용자 정의 모델로 객체 감지")
    print("📹 훈련된 모델에 따라 다양한 객체 감지 가능")
    print()
    print("🚀 사용 방법:")
    print("1. 코드 상단의 API_KEY, WORKSPACE, PROJECT 변수 수정")
    print("2. main_roboflow_detection() 실행")
    print()
    print("📝 또는 다음과 같이 사용:")
    print("setup_api_credentials('your_api_key', 'workspace', 'project')")
    print("=" * 60)

    # API 키를 여기에 직접 입력하거나
    # setup_api_credentials() 함수를 사용하세요

    main_roboflow_detection()

🎯 Roboflow YouTube 영상 객체 감지 시스템
💡 Roboflow 사용자 정의 모델로 객체 감지
📹 훈련된 모델에 따라 다양한 객체 감지 가능

🚀 사용 방법:
1. 코드 상단의 API_KEY, WORKSPACE, PROJECT 변수 수정
2. main_roboflow_detection() 실행

📝 또는 다음과 같이 사용:
setup_api_credentials('your_api_key', 'workspace', 'project')
📺 YouTube 영상 다운로드 시작...
[youtube] Extracting URL: https://www.youtube.com/watch?v=_CGb4GYHTvg
[youtube] _CGb4GYHTvg: Downloading webpage
[youtube] _CGb4GYHTvg: Downloading tv client config
[youtube] _CGb4GYHTvg: Downloading tv player API JSON
[youtube] _CGb4GYHTvg: Downloading ios player API JSON
[youtube] _CGb4GYHTvg: Downloading m3u8 information
[info] _CGb4GYHTvg: Downloading 1 format(s): 18
[download] ./input_video.mp4 has already been downloaded
[download] 100% of    2.02MiB
✅ YouTube 영상 다운로드 완료!
📁 영상 파일: input_video.mp4
🎯 Roboflow 모델 로드 중...
loading Roboflow workspace...
loading Roboflow project...
🎯 Roboflow 객체 감지 모델 로드 완료!
📁 프로젝트: jiyeonjin/0722_labeling-usrpl v1
🔍 객체 감지 시작...
🎬 영상 정보: 640x360, 30fps, 1305프레임
🎯 프레임 3: 2개 객체 감지
🎯 프레